In [1]:
import pandas as pd
import numpy as np
import matplotlib
import plotnine as ggplot
from plotnine import *
%matplotlib inline

import glob
import re

import time

from pandas import DataFrame
from IPython.display import HTML

import sys
sys.path.append("fastai/old/")
#from fastai_v7 import structured

import sys
sys.path.append("fastai/old/")
#from fastai_v7.structured import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer, OneHotEncoder
import os
import re
import glob
#os.environ["CUDA_VISIBLE_DEVICES"]="7"

from pandas import DataFrame
from IPython.display import HTML

import warnings
warnings.filterwarnings(action='once')
warnings.simplefilter('ignore')


In [2]:
filelist_loaded = []
filelist = glob.glob("/home/maghoi/pMHC_data/features5/*.csv"); len(filelist)

for i in range(0, len(filelist)):
    df = pd.read_csv(filelist[i])
    filelist_loaded.append(df)

In [3]:
#Extract names
name_list = []
for i in range(len(filelist_loaded)):
    r = re.compile("features5/(.*).csv")
    result = r.search(filelist[i])
    name_list.append(result.group(1))

In [4]:
def extract_sequence(dfs, i1, i2):
    seq_list = []
    for i in range(len(dfs)):
        seq = dfs[i].Sequence[i1 : i2]
        seq = np.where(seq == "-1", "-", seq)
        seq = "".join(seq)
        seq_list.append(seq)
    return(seq_list)

In [5]:
len(filelist_loaded[0].Sequence)

468

In [6]:
pep_list =  extract_sequence(filelist_loaded, 181, 192)
mhc_list = extract_sequence(filelist_loaded, 0, 181)
tcr_list = extract_sequence(filelist_loaded, 192, 468)

In [7]:
def extract_4th_col_row(df, axis):
    cols = []
    if axis == 1:
        for i in range(0, df.shape[1], 4):
            cols.append(i)
        return df.iloc[:, cols]
    
    if axis == 0:
        for i in range(0, df.shape[0], 4):
            cols.append(i)
        return df.iloc[cols]

In [8]:
#Remove 3/4 rows (replicates)
pep_list = list(extract_4th_col_row(pd.DataFrame(pep_list), 0)[0])
mhc_list = list(extract_4th_col_row(pd.DataFrame(mhc_list), 0)[0])
tcr_list = list(extract_4th_col_row(pd.DataFrame(tcr_list), 0)[0])

In [9]:
len(pep_list), len(mhc_list), len(tcr_list)

(366, 366, 366)

In [10]:
from Bio import pairwise2
def pairwise_align_score(seq_list):
    data = np.zeros((366,366))
    for i, p1 in enumerate(seq_list[:]):
        score_list = []

        for p2 in seq_list[:]:
            #p1 = p1.strip("-")
            #p2 = p2.strip("-")

            #Compute similarity
            a = pairwise2.align.globalms(p1, p2, 1, 0, -1, 0)[0][2]
            val = 0
            if len(p1) != len(p2):
                    val = 1
            score = (a+val) / max(len(p1), len(p2))
            score_list.append(score)


        #Append whole row to data
        data[i] = score_list
        if i % 1 == 0:
            print(i, "/", len(seq_list))

    #Create pandas dataframe with correct names
    df = pd.DataFrame(data, index = range(0, 366), columns = range(0, 366))
    return(df)


In [11]:
print("MHC pairwise align")
mhc_df = pairwise_align_score(mhc_list)
#mhc_df.to_feather("/home/maghoi/main/data/02_Features/MHCNames_Matrix.feather")



MHC pairwise align
0 / 366
1 / 366


KeyboardInterrupt: 

In [12]:
int_names = []
for i in range(0, 366, 1):
    int_names.append(str(i))
len(int_names)


366

In [ ]:
print("TCR pairwise align:")
tcr_df = pairwise_align_score(tcr_list)
tcr_df.columns = int_names
tcr_df.index = int_names
tcr_df.to_feather("/home/maghoi/main/data/02_Features/TCR_Matrix.feather")


#tcr_df.to_feather("/home/maghoi/main/data/02_Features/TCRNames_Matrix.feather")

TCR pairwise align:
0 / 366
1 / 366
2 / 366
3 / 366
4 / 366
5 / 366
6 / 366
7 / 366
8 / 366
9 / 366
10 / 366
11 / 366
12 / 366
13 / 366
14 / 366
15 / 366
16 / 366
17 / 366
18 / 366
19 / 366
20 / 366
21 / 366
22 / 366
23 / 366
24 / 366
25 / 366
26 / 366
27 / 366
28 / 366
29 / 366
30 / 366
31 / 366
32 / 366
33 / 366
34 / 366
35 / 366
36 / 366
37 / 366
38 / 366
39 / 366
40 / 366
41 / 366
42 / 366
43 / 366
44 / 366
45 / 366
46 / 366
47 / 366
48 / 366
49 / 366
50 / 366
51 / 366


In [ ]:
print("TCR pairwise align:")
tcr_df = pairwise_align_score(tcr_list)
tcr_df.columns = int_names
tcr_df.index = int_names
tcr_df.to_feather("/home/maghoi/main/data/02_Features/TCR_Matrix.feather")


#tcr_df.to_feather("/home/maghoi/main/data/02_Features/TCRNames_Matrix.feather")

TCR pairwise align:
0 / 366
1 / 366
2 / 366
3 / 366
4 / 366
5 / 366
6 / 366
7 / 366
8 / 366
9 / 366
10 / 366
11 / 366
12 / 366
13 / 366
14 / 366
15 / 366
16 / 366
17 / 366
18 / 366
19 / 366
20 / 366
21 / 366
22 / 366
23 / 366
24 / 366
25 / 366
26 / 366
27 / 366
28 / 366
29 / 366
30 / 366
31 / 366
32 / 366
33 / 366
34 / 366
35 / 366
36 / 366
37 / 366
38 / 366
39 / 366
40 / 366
41 / 366
42 / 366
43 / 366
44 / 366
45 / 366
46 / 366
47 / 366
48 / 366
49 / 366
50 / 366
51 / 366
52 / 366
53 / 366
54 / 366
55 / 366
56 / 366
57 / 366
58 / 366
59 / 366
60 / 366
61 / 366
62 / 366
63 / 366
64 / 366
65 / 366
66 / 366
67 / 366
68 / 366
69 / 366
70 / 366
71 / 366
72 / 366
73 / 366
74 / 366
75 / 366
76 / 366
77 / 366
78 / 366
79 / 366
80 / 366
81 / 366
82 / 366
83 / 366
84 / 366
85 / 366
86 / 366
87 / 366
88 / 366
89 / 366
90 / 366
91 / 366
92 / 366
93 / 366
94 / 366
95 / 366
96 / 366
97 / 366
98 / 366
99 / 366
100 / 366
101 / 366
102 / 366
103 / 366
104 / 366
105 / 366
106 / 366
107 / 366
108 / 366


In [ ]:
print("mhc pairwise align:")
mhc_df = pairwise_align_score(mhc_list)
mhc_df.columns = int_names
mhc_df.index = int_names
mhc_df.to_feather("/home/maghoi/main/data/02_Features/MHC_Matrix2.feather")

In [ ]:
2+2